In [53]:
import enum
import pandas as pd
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
cf.go_offline()

In [54]:
tradingList = {
    "HINDPETRO": {"price": 262.25, "leverage": 10},
    "HINDZINC": {"price": 317.10, "leverage": 10},
    "SBIN": {"price": 364.65, "leverage": 9},
    "ICICIBANK": {"price": 594.75, "leverage": 9},
    "WIPRO": {"price": 507.60, "leverage": 9},
    "ITC": {"price": 203.25, "leverage": 9},
    "BHARTIARTL": {"price": 561.75, "leverage": 9},
    "IOC": {"price": 102.80, "leverage": 9},
    "ONGC": {"price": 115.10, "leverage": 9},
    "DABUR": {"price": 529.20, "leverage": 9},
    "GAIL": {"price": 161.05, "leverage": 9},
    "TATACONSUM": {"price": 640.05, "leverage": 9},
    "AMBUJACEM": {"price": 309.05, "leverage": 9},
    "DLF": {"price": 265.30, "leverage": 8},
    "TATAPOWER": {"price": 109.35, "leverage": 8},
    "TATAMOTORS": {"price": 315.55, "leverage": 7}
}

tickerList = list(tradingList.keys())
data = pd.read_csv("E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\last_60\\17_05_2021\\days\\1.csv")

In [55]:
HINDPETRO = data[data["ticker"]=="HINDPETRO"]
HINDZINC = data[data["ticker"]=="HINDZINC"].reset_index().drop(['index', 'ticker'], axis=1)
SBIN = data[data["ticker"]=="SBIN"].reset_index().drop(['index', 'ticker'], axis=1)
ICICIBANK = data[data["ticker"]=="ICICIBANK"].reset_index().drop(['index', 'ticker'], axis=1)
WIPRO = data[data["ticker"]=="WIPRO"].reset_index().drop(['index', 'ticker'], axis=1)
ITC = data[data["ticker"]=="ITC"].reset_index().drop(['index', 'ticker'], axis=1)
IOC = data[data["ticker"]=="IOC"].reset_index().drop(['index', 'ticker'], axis=1)
ONGC = data[data["ticker"]=="ONGC"].reset_index().drop(['index', 'ticker'], axis=1)
DABUR = data[data["ticker"]=="DABUR"].reset_index().drop(['index', 'ticker'], axis=1)
GAIL = data[data["ticker"]=="GAIL"].reset_index().drop(['index', 'ticker'], axis=1)
TATACONSUM = data[data["ticker"]=="TATACONSUM"].reset_index().drop(['index', 'ticker'], axis=1)
AMBUJACEM = data[data["ticker"]=="AMBUJACEM"].reset_index().drop(['index', 'ticker'], axis=1)
DLF = data[data["ticker"]=="DLF"].reset_index().drop(['index', 'ticker'], axis=1)
TATAPOWER = data[data["ticker"]=="TATAPOWER"].reset_index().drop(['index', 'ticker'], axis=1)
TATAMOTORS = data[data["ticker"]=="TATAMOTORS"].reset_index().drop(['index', 'ticker'], axis=1)

In [56]:
def candle_graph(data,ticker):
    Candle = go.Candlestick(x=data.index,
                       open=data.open,
                       high=data.high,
                       low=data.low,
                       close=data.close
                       ) 
    iplot([Candle])

In [57]:
class Candle(enum.Enum):
    red = 1
    green = 2
    doji = 3    

In [58]:
class Trend(enum.Enum):
    strong_uptrend = 1
    uptrend = 2
    strong_downtrend = 3
    downtrend = 4
    uptrend_reversal = 5
    downtrend_reversal = 6
    strong_uptrend_reversal = 7
    strong_downtrend_reversal = 8
    sidetrend = 9  

In [59]:
def get_candle_details(data):
    candle_data = data.tail(4)
    current = get_last_candle(candle_data.tail(1))
    previous = get_last_candle(candle_data.tail(2).head(1))
    last_two = get_last_candle(candle_data.tail(3).head(1))
    last_three = get_last_candle(candle_data.head(1))
    return current,previous,last_two,last_three

In [60]:
def get_candle_data(data): 
    graph_data = pd.DataFrame(columns=['open','close','greenCandle','redCandle'])
    for index, row in data.iterrows():
        ticker_data = {}
        ticker_data['open'] = row['open'] 
        ticker_data['close'] = row['close'] 
        ticker_data['greenCandle'] = row['close'] > row['open']
        ticker_data['redCandle'] = row['close'] < row['open']
        ticker_data['doji'] = (row['close'] == row['open'])
        graph_data = graph_data.append(ticker_data, ignore_index=True)  
    return graph_data

In [61]:
def is_doji_candle(candle):
    return candle == Candle.doji

In [62]:
def is_green_candle(candle):
    return candle == Candle.green

In [63]:
def is_red_candle(candle):
    return candle == Candle.red

In [64]:
def get_last_candle(data):
    if data['close'].values > data['open'].values:
        return Candle.green
    if data['close'].values < data['open'].values:
        return Candle.red
    return Candle.doji     

In [75]:
def get_trends(data):
    current,previous,last_two,last_three = get_candle_details(data)
    if (is_green_candle(current) or is_doji_candle(current)) and (is_green_candle(previous) or is_doji_candle(previous)):
        if is_green_candle(last_two) or is_doji_candle(last_two):
            if is_green_candle(last_three) or is_doji_candle(last_three):
                trend = Trend.strong_uptrend
            else:
                trend = Trend.uptrend
    elif (is_red_candle(current) or is_doji_candle(current)) and (is_red_candle(previous) or is_doji_candle(previous)):
        if is_red_candle(last_two) or is_doji_candle(last_two):
            if is_red_candle(last_three) or is_doji_candle(last_three):
                trend = Trend.strong_downtrend
            else:
                trend = Trend.downtrend
    elif is_red_candle(current) and (is_green_candle(previous) or is_doji_candle(previous)):
        if is_green_candle(last_two) or is_doji_candle(last_two):
            if is_green_candle(last_three) or is_doji_candle(last_three):
                trend = Trend.strong_uptrend_reversal
            else:
                trend = Trend.uptrend_reversal
    elif is_green_candle(current) and (is_red_candle(previous) or is_doji_candle(previous)):
        if is_red_candle(last_two) or is_doji_candle(last_two):
            if is_red_candle(last_three) or is_doji_candle(last_three):
                trend = Trend.strong_downtrend_reversal
            else:
                trend = Trend.downtrend_reversal
    else:
        trend = Trend.sidetrend
    return trend

In [76]:
count = 12
_data = get_candle_data(HINDPETRO.head(count))
_data

,open,close,greenCandle,redCandle,doji
0,237.00,237.40,True,False,0.0
1,237.40,237.45,True,False,0.0
2,237.55,237.85,True,False,0.0
3,237.80,237.80,False,False,1.0
4,237.80,238.05,True,False,0.0
5,238.05,238.65,True,False,0.0
6,238.65,238.45,False,True,0.0
7,238.50,238.60,True,False,0.0
8,238.60,238.10,False,True,0.0
9,238.05,237.50,False,True,0.0


In [77]:
candle_graph(HINDPETRO.head(count),"HINDPETRO")

In [78]:

candle_data = HINDPETRO.head(7).tail(4)
_data = get_candle_data(candle_data)
_data

,open,close,greenCandle,redCandle,doji
0,237.80,237.80,False,False,1.0
1,237.80,238.05,True,False,0.0
2,238.05,238.65,True,False,0.0
3,238.65,238.45,False,True,0.0


In [79]:
candle_graph(candle_data,"HINDPETRO")

In [80]:
current,previous,last_two,last_three = get_candle_details(candle_data)
display("current",current)
display("previous",previous)
display("last_two",last_two)
display("last_three",last_three)    

'current'

<Candle.red: 1>

'previous'

<Candle.green: 2>

'last_two'

<Candle.green: 2>

'last_three'

<Candle.doji: 3>

In [81]:
trend = get_trends(candle_data)
trend     

<Trend.strong_uptrend_reversal: 7>

In [83]:
candle_data = HINDPETRO.head(count).tail(4)
candle_data

,open,high,low,close,volume,traded,startday,ticker
8,238.60,238.70,237.90,238.10,9478.0,False,2021-04-12-09:13:31,HINDPETRO
9,238.05,238.10,237.50,237.50,17075.0,False,2021-04-12-09:13:31,HINDPETRO
10,237.45,237.85,237.25,237.25,10597.0,False,2021-04-12-09:13:31,HINDPETRO
11,237.20,237.55,237.20,237.30,12706.0,False,2021-04-12-09:13:31,HINDPETRO


In [84]:
candle_graph(candle_data,"HINDPETRO")

In [85]:
trend = get_trends(candle_data)
trend 

<Trend.strong_downtrend_reversal: 8>

In [86]:
candle_data = HINDPETRO.head(count).head(4)
candle_data

,open,high,low,close,volume,traded,startday,ticker
0,237.00,237.75,235.90,237.40,38500.0,False,2021-04-12-09:13:31,HINDPETRO
1,237.40,237.70,237.00,237.45,42963.0,False,2021-04-12-09:13:31,HINDPETRO
2,237.55,237.95,237.15,237.85,39998.0,False,2021-04-12-09:13:31,HINDPETRO
3,237.80,238.00,237.25,237.80,58067.0,False,2021-04-12-09:13:31,HINDPETRO


In [87]:
candle_graph(candle_data,"HINDPETRO")

In [88]:
trend = get_trends(candle_data)
trend 

<Trend.strong_uptrend: 1>